In [1]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

using PyPlot
using Random

 Activating environment at `~/codes/hypergraph_modularities_code/Project.toml`


In [2]:
identity(p::Vector{Int64}) = p

function sum_of_ext_degs(p::Vector{Int64})
    soed = length(p) - 1
    return (sum(p), soed)
end

function all_or_nothing(p::Vector{Int64})
    is_aon = length(p) == 1
    return (sum(p), is_aon)
end

function rainbow(p::Vector{Int64})
    is_rainbow = length(p) == sum(p) && length(p) > 1
    return (sum(p), is_rainbow)
end

function all_or_nothing2(p::Vector{Int64})
    return length(p) == 1
end

all_or_nothing2 (generic function with 1 method)

In [3]:
function ll_ratios(Hyp, Z, agg, num_trials::Int64=100)
    function MLE_ll(Z1)
        Ω̂ = estimateΩEmpirically(Hyp, Z1; min_val=0, aggregator=agg)
        return Float64(sum(L(Hyp, Z1, Ω̂; α=0, bigInt=true)))    
    end
    data_ll = MLE_ll(Z)
    rand_ll = Float64[]
    for _ = 1:num_trials
        push!(rand_ll, MLE_ll(shuffle(Z)))
    end
    
    return data_ll, rand_ll
end

ll_ratios (generic function with 2 methods)

In [4]:
H, labels = read_hypergraph_data("contact-primary-school", 10)

(hypergraph
  N: Array{Int64}((242,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  233, 234, 235, 236, 237, 238, 239, 240, 241, 242]
  E: Dict{Int64,Dict}
  D: Array{Int64}((242,)) [236, 150, 183, 105, 147, 74, 49, 159, 222, 244  …  48, 141, 72, 114, 34, 34, 37, 84, 36, 102]
, [5, 5, 5, 5, 5, 6, 6, 6, 8, 8  …  4, 10, 2, 2, 6, 1, 9, 5, 10, 7])

In [5]:
data_ll, rand_ll = ll_ratios(H, labels, all_or_nothing2, 500)
data_ll .- rand_ll

500-element Array{Float64,1}:
 199923.0575860939
 520319.9448473833
 128923.92189508863
 235169.63439611928
 341968.91256951354
 148197.69011835242
 536622.2115340244
 238840.70668367273
 611855.6661032506
 532891.3509570444
 527806.0305207928
 272774.3054926202
 372686.13279928174
      ⋮
 401402.2485593322
 105008.09903349844
 284569.0343966475
 201229.804943525
 293890.55374597013
 422674.78275075904
 364410.3152967619
 503851.1858833055
 531569.4608265241
 420080.9257734136
 306877.5243821156
 288552.42352419253

In [6]:
H, labels = read_hypergraph_data("walmart-trips", 10)
data_ll, rand_ll = ll_ratios(H, labels, all_or_nothing2, 100)
data_ll .- rand_ll

100-element Array{Float64,1}:
 73245.66196355503
 72141.15182593837
 72146.6158016324
 72890.0711177215
 71325.11387897376
 70956.74057590123
 71367.55760686379
 72226.16261752415
 72659.71992012765
 74846.31414260995
 71668.30162708461
 72434.8749996284
 72144.402781819
     ⋮
 70390.50240353402
 72197.34548784234
 71532.70333244465
 73229.84461837634
 71076.55984119419
 75931.19257199578
 71464.58016780484
 69291.45854576211
 70790.28493395913
 71839.94194327481
 69758.19665842596
 71090.74127358291

In [7]:
H1, labels1 = read_hypergraph_data("TrivagoClickout", 5)
Ω̂ = estimateΩEmpirically(H1, labels1; min_val=0, aggregator=all_or_nothing)
ll1 = sum(L(H1, labels1, Ω̂; α=0, bigInt=true))

H2, labels2 = read_hypergraph_data("TrivagoClickout-cities", 5)
Ω̂ = estimateΩEmpirically(H2, labels2; min_val=0, aggregator=all_or_nothing)
ll2 = sum(L(H2, labels2, Ω̂; α=0, bigInt=true))

Float64(ll1 - ll2)

8.332846923148879e14